<a href="https://colab.research.google.com/github/antounes/deep-learning-python/blob/main/3_4_classifying_movie_reviews_a_binary_classification_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying Movie Reviews: A Binary Classification Example

In [23]:
import tensorflow as tf
import numpy as np

## The IMBD Dataset

In [24]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000) # Keep only the 10,000 most frequently occurring words in the training data

In [25]:
#train_data[0]
#train_labels[0]

In [26]:
# Let's quickly decode one of the reviews back to English vocabulary

word_index = imdb.get_word_index() # dictionary mapping words to an integer index
reverse_word_index = {value: key for key, value in word_index.items()}
decoded_review = ' '.join(
    [reverse_word_index.get(i-3, '?') for i in train_data[0]] # indices offset by 3 because 0, 1 and 2 are reserved indices for "padding", "start of sequence" and "unknown"
)

In [27]:
decoded_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

## Preparing the data

In [28]:
# Encoding the integer sequences into a binary matrix

def vectorize_sequences(sequences, dimension=10000):
  results = np.zeros((len(sequences), dimension))  # create and all-zero matrix of shape (len(sequences), dimension)
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1. # set specific indices of results[i] to 1s
  return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [29]:
# Vetorise labels

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [30]:
y_train

array([1., 0., 0., ..., 0., 1., 0.], dtype=float32)

## Building the Network

In [31]:
# Input data = vectors
# Output data = scalars
# A type of network that performs well on such a problem is a simple stack of fully connected (`Dense`) layers with `relu` activations: `Dense(16, activation='relu')`
# Two intermediate layers with 16 hidden units each with `relu`activation
# A third layer that will output the scalar prediction regarding the sentiment of the current review with `sigmoid` activation (outputs a probability between 0 and 1 indicating
# how likely the sample is to have the target `1`: i.e. how likely the review is to be positive)

model = tf.keras.models.Sequential([
     tf.keras.layers.Flatten(input_shape=(10000,)),
     tf.keras.layers.Dense(16, activation='relu'),
     tf.keras.layers.Dense(16, activation='relu'),
     tf.keras.layers.Dense(1, activation='sigmoid')])

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Validating the approach

In [33]:
# Set aside a validation set

x_val, y_val = x_train[:10000], y_train[:10000]
partial_x_train, partial_y_train = x_train[10000:], y_train[10000:]

In [ ]:
# Train the model for 20 epochs (20 iterations over all samples in the x_train and y_train tensors)
# Mini-batches of 512 samples
# Monitor accuracy at the same time on the 10,000 samples set apart by passing the validation data as the `validation_data` argument

history = model.fit(
    partial_x_train, partial_y_train, epochs=20, batch_size=512,  validation_data=(x_val, y_val))